# JUICE RPWI HF SID23 (PSSR3 rich): L1a QL -- 2025/10/14

# Import lib

In [ ]:
import copy
import sys
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Library setting

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib   as juice_cal
import juice_cdf_lib   as juice_cdf
# import juice_sid23_data as juice_data
import juice_sid23_lib as juice_sid23
import juice_spec_lib  as juice_spec

# Mode seting

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump
# *** CAL set ***
unit_mode = 0                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 1                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background   1: CAL     2: all
# *** Unit set ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 0                           # [Time]      0: Epoch   1: data number
gap_mode  = 0                           # [gap]       0: n/a     1: gap
# *** Parameter set ***
p_raw_max = 7.0                         # background: 7.5
p_raw_min = 0.0                         # background: 2.5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
RIME_mode = 0                           # [RIME rejection]  0: n/a  1: RIME reject
RFI_mode  = 0                           # [RFI  rejection]  0: n/a  1: 200kHz reject
f_min     = 0000                        # [Frequency sel]   max - min
f_max     = 45000                       # [Frequency sel]   max - min

# get CDF data

In [ ]:
# *** Flight data: Ver.2 ***
# date='20240126';  ver = 'V02'
# date='20240706';  ver = 'V02'
# date='20240819';  ver = 'V02'
# date='20240821';  ver = 'V02'

# *** Ground test data ***
date='0';  ver = 'V02'

data_dir, data_name_list = juice_sid23.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    # RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid23.hf_sid23_read(cdf) #, RPWI_FSW_version)
    if i==0:
        data = data1;                                  print(data.Eu_i.shape)
    else:
        data = juice_sid23.hf_sid23_add(data, data1);  print(data.Eu_i.shape)
data_name = os.path.split(data_name)[1];               print("data name:", data_name)

In [ ]:
data = juice_sid23.hf_sid23_shaping(data, f_max, f_min)
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.n_time;  n_block0 = data.time.shape[1];  n_length0 = data.time.shape[2]
n_num0  = n_block0 * n_length0 

In [ ]:
if data.n_time>1: 
    date3 = data.epoch[1];  date3 = date3.strftime('%Y/%m/%d %R:%S')
    print("       date and time:", str_date, "(interval:", data.epoch[1] - data.epoch[0], date3, ")")
else:
    print("       date and time:", str_date)
print("           data size:", data.Eu_i.shape, "  [ Num-samples:", n_time0, "  Num-Block:", n_block0, " * Length:", n_length0, " = Num:", n_num0, "]")
print("         time_length:", data.time.shape, "  [", data.time[0][0][n_length0-1], data.time[0][1][0], "sec in 1-block", data.time[0][n_block0-1][n_length0-1], "sec in total")

### RIME rejection ---- "Continuous pulse with Width = 81 (0.28msec)"

In [ ]:
# data = juice_sid23.hf_sid23_rime_reject(data)
if RIME_mode>0:
    data.EuEu = data.Eu_i * data.Eu_i + data.Eu_q * data.Eu_q
    data.EvEv = data.Ev_i * data.Ev_i + data.Ev_q * data.Ev_q
    data.EwEw = data.Ew_i * data.Ew_i + data.Ew_q * data.Ew_q
    data.EE   = copy.copy(data.EuEu)
    data.EE_flag = copy.copy(data.EuEu)
    thres = 10

    for i in range(data.n_time):
        data.EE[i] = data.EuEu[i] + data.EvEv[i] + data.EwEw[i]
                    #  * data.U_selected[i] + 

        for j in range(data.n_block):
            EE_med = np.median(data.EE[i][j]);  EE_peak = max(data.EE[i][j])
            index = np.where(data.EE[i][j] <  EE_med * thres);  data.EE_flag[i][j][index] = 0   # EE_med
            index = np.where(data.EE[i][j] >= EE_med * thres);  data.EE_flag[i][j][index] = 1   # EE_peak1 

            # Short off (<40): treated as ON
            k_off = 0
            for k in range(data.n_feed*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        if k - k_off < 40:
                            data.EE_flag[i][j][k_off:k] = data.EE_flag[i][j][k]
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # OFF
                        k_off = k

            """
            # Short on (<400: treated as OFF
            for k in range(data.n_feed*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # off
                        if k - k_on < 40:
                            data.EE_flag[i][j][k_on:k] = data.EE_flag[i][j][k]
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        k_on = k
            """

            # DIS before/after ON
            k_on = 0
            k_off = 0
            for k in range(data.n_feed*128):
                if k > 0:
                    if data.EE_flag[i][j][k-1] < data.EE_flag[i][j][k]:     # ON
                        k_on = k
                    if data.EE_flag[i][j][k-1] > data.EE_flag[i][j][k]:     # OFF
                        # if i==8 and j==0:
                        #    print(k, k_off, k_on)
                        if k_on -8 > 0:
                            data.EE_flag[i][j][k_on-8:k_on]   = 1
                        if k_off+8 < data.n_feed*128:
                            data.EE_flag[i][j][k_off:k_off+8] = 1
                        k_off = k
            if k_off!=0 and k_off+10 < data.n_feed*128:
                data.EE_flag[i][j][k_off:k_off+10] = 1


    index = np.where(data.EE_flag > 0)
    # data.EE[index] = EE_med
    data.Eu_i2 = copy.copy(data.Eu_i);  data.Eu_q2 = copy.copy(data.Eu_q)
    data.Ev_i2 = copy.copy(data.Ev_i);  data.Ev_q2 = copy.copy(data.Ev_q)
    data.Ew_i2 = copy.copy(data.Ew_i);  data.Ew_q2 = copy.copy(data.Ew_q)
    data.Eu_i2[index] = 0;  data.Eu_q2[index] = 0
    data.Ev_i2[index] = 0;  data.Ev_q2[index] = 0
    data.Ew_i2[index] = 0;  data.Ew_q2[index] = 0
    data.Eu_i[index] = 0;  data.Eu_q[index] = 0
    data.Ev_i[index] = 0;  data.Ev_q[index] = 0
    data.Ew_i[index] = 0;  data.Ew_q[index] = 0

    #
    # ---- TEST PLOT ----
    #
    n_sweep = 1
    n_block = 9 # n_block0
    print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.n_block)

    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

    """
    ax1.plot(np.ravel(data.EE_flag[n_sweep][n_block]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE_flag[n_sweep][n_block]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     [n_sweep][n_block]), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  [n_sweep][n_block]), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  [n_sweep][n_block]), '-r', linewidth=.5, label='Ew_q')
    """
    """
    ax1.plot(np.ravel(data.EE_flag[n_sweep]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE_flag[n_sweep]), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     [n_sweep]), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  [n_sweep]), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  [n_sweep]), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  [n_sweep]), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  [n_sweep]), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  [n_sweep]), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  [n_sweep]), '-r', linewidth=.5, label='Ew_q')
    """
    ax1.plot(np.ravel(data.EE_flag), '-k', linewidth=1,  label='EE_flag');  # TMP
    ax1.plot(np.ravel(data.EE     ), '-r', linewidth=1,  label='EE');          # TMP
    ax2.plot(np.ravel(data.Eu_i2  ), '-b', linewidth=.5, label='Eu_i'); 
    ax2.plot(np.ravel(data.Eu_q2  ), '-r', linewidth=.5, label='Eu_q')
    ax3.plot(np.ravel(data.Ev_i2  ), '-b', linewidth=.5, label='Ev_i'); 
    ax3.plot(np.ravel(data.Ev_q2  ), '-r', linewidth=.5, label='Ev_q')
    ax4.plot(np.ravel(data.Ew_i2  ), '-b', linewidth=.5, label='Ew_i'); 
    ax4.plot(np.ravel(data.Ew_q2  ), '-r', linewidth=.5, label='Ew_q')

    ax2.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax4.set_ylabel('Ew')
    ax4.set_xlabel('Time (sec)')
    #
    title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]\n' + data_name + "\n[" + date1 + "]";  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax4.legend(loc='upper right', fontsize=8)

    # range: X-axis
    """
    # xlim=[0.00159, 0.00187]   # 0.28msec
    xlim=[1300, 1420]             # 81 * 3.37837832e-06 = 0.28msec
    ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
    # range: Y-axis
    ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
    ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim);  ax4.set_ylim(ylim)
    """
    ax1.set_yscale('log')

    fig.subplots_adjust(hspace=0)
    fig.show

### RFI noise rejection

In [ ]:
if RFI_mode > 0:
    from scipy import signal
    dt = 1.0 / juice_cdf._sample_rate(data.decimation_AUX[0])
    f_cut = 5000                   # Cutoff: 10kHz
    f_tap = 129
    filter = signal.firwin(numtaps=f_tap, cutoff=f_cut, fs=1/dt, pass_zero=False)

    print("frequency cutoff(Hz):", f_cut, "   delay:", dt * f_tap )
    for i in range(data.n_time):
        for j in range(data.n_block):
            data.Eu_i[i][j] = signal.lfilter(filter, 1, data.Eu_i[i][j]);  data.Eu_q[i][j] = signal.lfilter(filter, 1, data.Eu_q[i][j])
            data.Ev_i[i][j] = signal.lfilter(filter, 1, data.Ev_i[i][j]);  data.Ev_q[i][j] = signal.lfilter(filter, 1, data.Ev_q[i][j])
            data.Ew_i[i][j] = signal.lfilter(filter, 1, data.Ew_i[i][j]);  data.Ew_q[i][j] = signal.lfilter(filter, 1, data.Ew_q[i][j])

## CAL Waveform

In [ ]:
T_HF  = 25;  T_RWI = 25
wave = juice_cal.wave_cal(data, 23, unit_mode, T_HF, T_RWI)
p_max0 = p_raw_max + wave.cf/10;     p_min0 = p_raw_min + wave.cf/10

print("unit_mode:", unit_mode, "  conversion factor:", '{:.1f}'.format(wave.cf), 
      "  MAX-min:", '{:.1f}'.format(p_max0), '{:.1f}'.format(p_min0), "  str_unit:", wave.str_unit)

In [ ]:
# print("decimation:", juice_cdf._sample_rate(data.decimation[0]))
print("Peak Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(np.ravel(wave.Eu_i[:][:]), '-b', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(wave.Eu_q[:][:]), ':r', linewidth=.1, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[:][:]), '-b', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(wave.Ev_q[:][:]), ':r', linewidth=.1, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[:][:]), '-b', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(wave.Ew_q[:][:]), ':r', linewidth=.1, label='Ew_q')
ax4.plot(np.ravel(data.time[:]), linewidth=.5, label='time')
ax5.plot(np.ravel(data.freq_center[:]), '.')
ax5.plot(np.ravel(data.HF_QF*1000),   '.r', label='HF_QF')
ax6.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);    ax3.set_ylabel('Ew '+wave.str_unit);  
ax4.set_ylabel('Time [sec]');    ax5.set_ylabel('f_center [kHz]');  ax6.set_xlabel(str_date)
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.Eu_i)) -.5];  print(xlim);  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5];  print(xlim);  ax5.set_xlim(xlim);  ax6.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

### WAVE - 1 Sweep

In [ ]:
n_sweep = n_time0//2
n_block = n_block0
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0, "   Block:", n_block, "in total of", data.n_block)

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4);  

ax1.plot(np.ravel(wave.Eu_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Eu_i');  ax1.plot(np.ravel(wave.Eu_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ev_i');  ax2.plot(np.ravel(wave.Ev_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ew_i');  ax3.plot(np.ravel(wave.Ew_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.time[n_sweep][0:n_block]), linewidth=.5, label='time')

ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  ax4.set_ylabel('Time [sec]')
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]\n' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)
date = data.epoch[n_sweep];  date = date.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date)

xlim=[-0.5, n_block*data.n_feed*128                   -.5];  print(xlim);  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim)
xlim=[-0.5, len(np.ravel(data.time[n_sweep][0:n_block])) -.5];  print(xlim);  ax4.set_xlim(xlim)
# ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
# ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

print(len(np.ravel(data.time[n_sweep][0:n_block])))

### WAVE - 1 block

In [ ]:
n_sweep = n_time0//2
n_block = 0
print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.n_block)

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(data.time[n_sweep][n_block], wave.Eu_i[n_sweep][n_block], '-b', linewidth=.5, label='Eu_i'); 
ax1.plot(data.time[n_sweep][n_block], wave.Eu_q[n_sweep][n_block], '-r', linewidth=.5, label='Eu_q')
ax2.plot(data.time[n_sweep][n_block], wave.Ev_i[n_sweep][n_block], '-b', linewidth=.5, label='Ev_i'); 
ax2.plot(data.time[n_sweep][n_block], wave.Ev_q[n_sweep][n_block], '-r', linewidth=.5, label='Ev_q')
ax3.plot(data.time[n_sweep][n_block], wave.Ew_i[n_sweep][n_block], '-b', linewidth=.5, label='Ew_i'); 
ax3.plot(data.time[n_sweep][n_block], wave.Ew_q[n_sweep][n_block], '-r', linewidth=.5, label='Ew_q')
ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit)
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]\n' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)
date = data.epoch[n_sweep];  date = date.strftime('%Y/%m/%d %R:%S') + "    time [sec]  (0.001sec delay = 150-km distance)";  ax3.set_xlabel(date)

xlim=[data.time[n_sweep][n_block][0], data.time[n_sweep][n_block][-1]];  print(xlim);  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim)
# xlim=[0.00, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim)   # 0.28msec
# xlim=[0.00159, 0.00187]   # 0.28msec
# xlim=[334, 415]             # 81 * 3.37837832e-06 = 0.28msec
# ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)];  ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-block.png'
    fig.savefig(png_fname)

# Spectrum Data

In [ ]:
data.time[0][0]

In [ ]:
spec = juice_spec.hf_getspec_sid23(data)

In [ ]:
n_time  = spec.EuEu.shape[0];  n_block = spec.EuEu.shape[1];  n_freq  = spec.EuEu.shape[2]

spec.freq    = np.array(spec.freq).reshape   (n_time * n_block, n_freq)
spec.freq_w  = np.array(spec.freq_w).reshape (n_time * n_block, n_freq)
spec.EuEu    = np.array(spec.EuEu).reshape   (n_time * n_block, n_freq)
spec.EvEv    = np.array(spec.EvEv).reshape   (n_time * n_block, n_freq)
spec.EwEw    = np.array(spec.EwEw).reshape   (n_time * n_block, n_freq)
spec.EuEv_re = np.array(spec.EuEv_re).reshape(n_time * n_block, n_freq)
spec.EvEw_re = np.array(spec.EvEw_re).reshape(n_time * n_block, n_freq)
spec.EwEu_re = np.array(spec.EwEu_re).reshape(n_time * n_block, n_freq)
spec.EuEv_im = np.array(spec.EuEv_im).reshape(n_time * n_block, n_freq)
spec.EvEw_im = np.array(spec.EvEw_im).reshape(n_time * n_block, n_freq)
spec.EwEu_im = np.array(spec.EwEu_im).reshape(n_time * n_block, n_freq)
#
spec = juice_cal.spec_cal(spec, 23, unit_mode, band_mode, T_HF, T_RWI)   # CAL
#
spec.freq    = np.array(spec.freq).reshape   (n_time, n_block, n_freq)
spec.freq_w  = np.array(spec.freq_w).reshape (n_time, n_block, n_freq)
spec.EuEu    = np.array(spec.EuEu).reshape   (n_time, n_block, n_freq)
spec.EvEv    = np.array(spec.EvEv).reshape   (n_time, n_block, n_freq)
spec.EwEw    = np.array(spec.EwEw).reshape   (n_time, n_block, n_freq)
spec.EuEv_re = np.array(spec.EuEv_re).reshape(n_time, n_block, n_freq)
spec.EvEw_re = np.array(spec.EvEw_re).reshape(n_time, n_block, n_freq)
spec.EwEu_re = np.array(spec.EwEu_re).reshape(n_time, n_block, n_freq)
spec.EuEv_im = np.array(spec.EuEv_im).reshape(n_time, n_block, n_freq)
spec.EvEw_im = np.array(spec.EvEw_im).reshape(n_time, n_block, n_freq)
spec.EwEu_im = np.array(spec.EwEu_im).reshape(n_time, n_block, n_freq)
spec.EE      = spec.EuEu + spec.EvEv + spec.EwEw 

p_max1 = p_raw_max + spec.cf/10; p_min1 = p_raw_min + spec.cf/10


In [ ]:
# spec = juice_spec.hf_getspec_angle    (spec, 23)     # Coherency & ange
# spec = juice_spec.hf_getspec_stokes   (spec, 23)     # Stokes and Polarization Parameters 

In [ ]:
n_sweep = np.int16(n_time0/2)
n_block = 0
print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.n_block)

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)

wave.EE = wave.Eu_i*wave.Eu_i + wave.Eu_q*wave.Eu_q # + wave.Ev_i*wave.Ev_i + wave.Ev_q*wave.Ev_q + wave.Ew_i*wave.Ew_i + wave.Ew_q*wave.Ew_q
ax1.plot(data.time[n_sweep][n_block], wave.EE[n_sweep][n_block], '-b', linewidth=.5, label='EE')
ax1.set_ylabel('EE ' +spec.str_unit)
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]\n' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8)
ax1.set_yscale('log')
date = data.epoch[n_sweep];  date = date.strftime('%Y/%m/%d %R:%S') + "    time [sec]  (0.001sec delay = 150-km distance)";  ax1.set_xlabel(date)

xlim=[data.time[n_sweep][n_block][0], data.time[n_sweep][n_block][-1]];  
# xlim=[0.000, 0.0033]   # 0.28msec
print(xlim);  ax1.set_xlim(xlim)
# xlim=[334, 415]           # 81 * 3.37837832e-06 = 0.28msec
# ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]; ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-block.png'
    fig.savefig(png_fname)

In [ ]:
print(" data shape:", spec.EE.shape, n_time0, n_block0, n_length0)
print("freq center:", data.freq_center)
print("unit_mode:", unit_mode, "  conversion factor:", '{:.1f}'.format(spec.cf), "[wave:", '{:.1f}'.format(wave.cf),
      "]  MAX-min:", '{:.1f}'.format(p_max1), '{:.1f}'.format(p_min1), ")")

In [ ]:
print("Peak   Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak   Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak   Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)
peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)

In [ ]:
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
print(f"[First peak - {n_sweep1}]");  freq_1d = spec.freq[n_sweep1][0]
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
print(f"[Mid   peak - {n_sweep2}]");  freq_1d = spec.freq[n_sweep2][0]
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
print(f"[Last  peak - {n_sweep3}]");  freq_1d = spec.freq[n_sweep3][0]
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, ' {:.1f} kHz'.format(freq_1d[peak_f%freq_1d.shape[0]]))

# Spectrum - first

In [ ]:
n_sweep = 0;  print("[first]  SWEEP:", n_num0, "in total of", n_time0)
p_min = p_min1-2;  p_max = p_max1
f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-r', linewidth=.5, label='EuEu')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE '+spec.str_unit); ax4.set_ylabel('EuEu '+spec.str_unit); ax5.set_ylabel('EvEv '+spec.str_unit); ax6.set_ylabel('EwEw '+spec.str_unit); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S'); title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)

ylim=[10**(p_min), 10**(p_max)]; ax1.set_ylim(ylim); ax4.set_ylim(ylim);  ax5.set_ylim(ylim); ax6.set_ylim(ylim)

fig.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-first.png'
    fig.savefig(png_fname)

peak_E = np.ravel(spec.EE  [n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")

## Spectrum - Mid

In [ ]:
n_sweep = n_time0//2;  print("[mid]  SWEEP:", n_num0, "in total of", n_time0)
p_min = p_min1-2;  p_max = p_max1
f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-r', linewidth=.5, label='EuEu')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE '+spec.str_unit); ax4.set_ylabel('EuEu '+spec.str_unit); ax5.set_ylabel('EvEv '+spec.str_unit); ax6.set_ylabel('EwEw '+spec.str_unit); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S'); title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)

ylim=[10**(p_min), 10**(p_max)]; ax1.set_ylim(ylim); ax4.set_ylim(ylim);  ax5.set_ylim(ylim); ax6.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-mid.png'
    fig.savefig(png_fname)

peak_E = np.ravel(spec.EE  [n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")

## Spectrum - Last

In [ ]:
n_sweep = n_time0-1;  print("[last]  SWEEP:", n_num0, "in total of", n_time0)
p_min = p_min1-2;  p_max = p_max1
# p_min = -12.0;    p_max = -6.0
f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-r', linewidth=.5, label='EuEu')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax1.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax4.plot(spec.freq[n_sweep][0], np.ravel(spec.EuEu[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq[n_sweep][0], np.ravel(spec.EvEv[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][0][:]),          '-r', linewidth=.5, label='first')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq[n_sweep][0], np.ravel(spec.EwEw[n_sweep][n_block-1][:]),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE '+spec.str_unit); ax4.set_ylabel('EuEu '+spec.str_unit); ax5.set_ylabel('EvEv '+spec.str_unit); ax6.set_ylabel('EwEw '+spec.str_unit); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
ax1.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S'); title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)

ylim=[10**(p_min), 10**(p_max)]; ax1.set_ylim(ylim); ax4.set_ylim(ylim);  ax5.set_ylim(ylim); ax6.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-last.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(spec.EE  [n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, " at", '{:.1f}'.format(spec.freq[n_sweep][0][peak_f]), "kHz")

# Frequency - Time diagram

In [ ]:
n_sweep = 0;  f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]
p_min = p_min1-2;  p_max = p_max1

freq_1d = spec.freq[n_time0//2][0]
n_num = n_time0 * n_block0
EE_2d   = np.ravel(spec.EE);   EE_2d   = EE_2d.reshape  (n_num, n_length0).transpose()
EuEu_2d = np.ravel(spec.EuEu); EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(spec.EvEv); EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(spec.EwEw); EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11]);  
gs = gridspec.GridSpec(4,1, height_ratios=[1,1,1,.5]) 
ax1 = fig2d.add_subplot(gs[0]);  ax2 = fig2d.add_subplot(gs[1]); ax3 = fig2d.add_subplot(gs[2]); ax4 = fig2d.add_subplot(gs[3])
ax1.set_ylim(f_min, f_max);    ax2.set_ylim(f_min, f_max);    ax3.set_ylim(f_min, f_max)
ax1.set_ylabel('EuEu [kHz]');  ax2.set_ylabel('EvEv [kHz]');  ax3.set_ylabel('EwEw [kHz]')
ax1.set_title("[" + data_name + "]");                         ax3.set_xlabel(str_date)

"""
p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
"""
p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(spec.str_unit)
pp4 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp4.set_label(spec.str_unit)

xlim=[-.5, len(np.ravel(data.epoch))-.5];  print(xlim);  ax4.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-FT.png'
    fig2d.savefig(png_fname)

In [ ]:
n_sweep = 0;  f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]
p_min = p_min1-2;  p_max = p_max1
p_min = -15.5;       p_max = -12

freq_1d = spec.freq[n_time0//2][0]
#
n_num = n_time0 * n_block0
EE_2d   = np.ravel(spec.EE);   EE_2d   = EE_2d.reshape  (n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11]); ax1 = fig2d.add_subplot(1, 1, 1)
ax1.set_ylim(f_min, f_max);  ax1.set_ylabel('EE [kHz]')
ax1.set_title("[" + data_name + "]");                         ax3.set_xlabel(str_date)

p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d[:], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(spec.str_unit)

plt.subplots_adjust(hspace=0.02)
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-FT-EE.png'
    fig2d.savefig(png_fname)

In [ ]:
n_sweep = 0;  f_min = spec.freq[n_sweep][0][0];  f_max = spec.freq[n_sweep][0][-1]; d_freq = spec.freq[n_sweep][0][1]-spec.freq[n_sweep][0][0]
p_min = p_min1-2;  p_max = p_max1
#
n_num = n_time0 * n_block0
E_DoLuv_2d = np.ravel(spec.E_DoLuv); E_DoLuv_2d = E_DoLuv_2d.reshape(n_num, n_length0).transpose()
E_DoLvw_2d = np.ravel(spec.E_DoLvw); E_DoLvw_2d = E_DoLvw_2d.reshape(n_num, n_length0).transpose()
E_DoLwu_2d = np.ravel(spec.E_DoLwu); E_DoLwu_2d = E_DoLwu_2d.reshape(n_num, n_length0).transpose()
E_DoCuv_2d = np.ravel(spec.E_DoCuv); E_DoCuv_2d = E_DoCuv_2d.reshape(n_num, n_length0).transpose()
E_DoCvw_2d = np.ravel(spec.E_DoCvw); E_DoCvw_2d = E_DoCvw_2d.reshape(n_num, n_length0).transpose()
E_DoCwu_2d = np.ravel(spec.E_DoCwu); E_DoCwu_2d = E_DoCwu_2d.reshape(n_num, n_length0).transpose()
E_ANGuv_2d = np.ravel(spec.E_ANGuv); E_ANGuv_2d = E_ANGuv_2d.reshape(n_num, n_length0).transpose()
E_ANGvw_2d = np.ravel(spec.E_ANGvw); E_ANGvw_2d = E_ANGvw_2d.reshape(n_num, n_length0).transpose()
E_ANGwu_2d = np.ravel(spec.E_ANGwu); E_ANGwu_2d = E_ANGwu_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(9, 1, 1); ax2 = fig2d.add_subplot(9, 1, 2); ax3 = fig2d.add_subplot(9, 1, 3); ax4 = fig2d.add_subplot(9, 1, 4); ax5 = fig2d.add_subplot(9, 1, 5)
ax6 = fig2d.add_subplot(9, 1, 6); ax7 = fig2d.add_subplot(9, 1, 7); ax8 = fig2d.add_subplot(9, 1, 8); ax9 = fig2d.add_subplot(9, 1, 9)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max)
ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max); ax6.set_ylim(f_min, f_max)
ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
ax1.set_ylabel('DoL-UV'); ax2.set_ylabel('DoL-VW'); ax3.set_ylabel('DoL-WU')
ax4.set_ylabel('DoC-UV'); ax5.set_ylabel('DoC-VW'); ax6.set_ylabel('DoC-WU')
ax7.set_ylabel('Ang-UV'); ax8.set_ylabel('Ang-VW'); ax9.set_ylabel('Ang-WU')
ax1.set_title("[" + data_name + "]");  ax9.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical")
pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical")
pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")

plt.subplots_adjust(hspace=0.03)
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_AngT-FT.png'
    fig2d.savefig(png_fname)

# Get Auto Correlation

In [ ]:
auto = juice_sid23.hf_sid23_getauto(data)

print(auto.EuEu.shape, n_time0, n_block0, n_length0)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

## Auto First

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE autocorr');  ax2.set_ylabel('EuEu autocorr'); ax3.set_ylabel('EvEv autocorr'); ax4.set_ylabel('EwEw autocorr'); ax4.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# xlim=[0, 0.0001];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')

plt.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-1st.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.nanargmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Mid

In [ ]:
n_sweep = np.int16(n_time0//2)
print("[mid]  SWEEP:", n_num0, "in total of", n_time0)

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE autocorr');  ax2.set_ylabel('EuEu autocorr'); ax3.set_ylabel('EvEv autocorr'); ax4.set_ylabel('EwEw autocorr'); ax4.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')

plt.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-mid.png'
    fig.savefig(png_fname)

peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.nanargmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Last

In [ ]:
n_sweep = n_time0 - 1

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][0][:])),          '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block//2][:])), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(abs(auto.EE  [n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(abs(auto.EuEu[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(abs(auto.EvEv[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(abs(auto.EwEw[n_sweep][n_block-1][:])),  '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE autocorr');  ax2.set_ylabel('EuEu autocorr'); ax3.set_ylabel('EvEv autocorr'); ax4.set_ylabel('EwEw autocorr'); ax4.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log')

# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')

plt.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-last.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.nanargmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Time

In [ ]:
n_num = n_time0 * n_block0
p_min = 1e-4;  p_max =  1/4
EE_2d   = np.ravel(auto.EE);    EE_2d   = EE_2d.reshape(n_num, n_length0).transpose()
EuEu_2d = np.ravel(auto.EuEu);  EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(auto.EvEv);  EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(auto.EwEw);  EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3);  ax4 = fig2d.add_subplot(4, 1, 4)

ax1.set_ylabel('EE [sec]');  ax2.set_ylabel('EuEu [sec]');  ax3.set_ylabel('EvEv [sec]');  ax4.set_ylabel('EwEw [sec]')
ax1.set_title("Autocorr [" + data_name + "]");  ax4.set_xlabel(str_date)

# Plot: linear
#p1 = ax1.pcolormesh(num_1d, data.time[0][0], EE_2d[:],   vmin=p_min, vmax=p_max, cmap='bwr')
#p2 = ax2.pcolormesh(num_1d, data.time[0][0], EuEu_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
#p3 = ax3.pcolormesh(num_1d, data.time[0][0], EvEv_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
#p4 = ax4.pcolormesh(num_1d, data.time[0][0], EwEw_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
p1  = ax1.pcolormesh(num_1d, data.time[0][0], abs(EE_2d[:]),   norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p2  = ax2.pcolormesh(num_1d, data.time[0][0], abs(EuEu_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p3  = ax3.pcolormesh(num_1d, data.time[0][0], abs(EvEv_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p4  = ax4.pcolormesh(num_1d, data.time[0][0], abs(EwEw_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('auto-corr')
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label('auto-corr')
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label('auto-corr')
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical");  pp4.set_label('auto-corr')

plt.subplots_adjust(hspace=0.02); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-FT.png'
    fig2d.savefig(png_fname)

In [ ]:
n_num = n_time0 * n_block0
p_min = 1e-4;  p_max =  1/4
EE_2d   = np.ravel(auto.EE);    EE_2d   = EE_2d.reshape(n_num, n_length0).transpose()
EuEu_2d = np.ravel(auto.EuEu);  EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(auto.EvEv);  EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(auto.EwEw);  EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3);  ax4 = fig2d.add_subplot(4, 1, 4)

ax1.set_ylabel('EE [km]');  ax2.set_ylabel('EuEu [km]');  ax3.set_ylabel('EvEv [km]');  ax4.set_ylabel('EwEw [km]')
ax1.set_title("Autocorr [" + data_name + "]");  ax4.set_xlabel(str_date)

#p1 = ax1.pcolormesh(num_1d, data.time[0][0]*3e5/2, EE_2d[:],   vmin=p_min, vmax=p_max, cmap='bwr')
#p2 = ax2.pcolormesh(num_1d, data.time[0][0]*3e5/2, EuEu_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
#p3 = ax3.pcolormesh(num_1d, data.time[0][0]*3e5/2, EvEv_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
#p4 = ax4.pcolormesh(num_1d, data.time[0][0]*3e5/2, EwEw_2d[:], vmin=p_min, vmax=p_max, cmap='bwr')
p_min = 1e-4;  p_max =  1/4
p1  = ax1.pcolormesh(num_1d, data.time[0][0]*3e5/2, abs(EE_2d[:]),   norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p2  = ax2.pcolormesh(num_1d, data.time[0][0]*3e5/2, abs(EuEu_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p3  = ax3.pcolormesh(num_1d, data.time[0][0]*3e5/2, abs(EvEv_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
p4  = ax4.pcolormesh(num_1d, data.time[0][0]*3e5/2, abs(EwEw_2d[:]), norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('auto-corr')
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label('auto-corr')
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label('auto-corr')
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical");  pp4.set_label('auto-corr')

plt.subplots_adjust(hspace=0.02); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-FT-km.png'
    fig2d.savefig(png_fname)

In [ ]:
n_num  = n_time0 * n_block0
p_min = 1e-4;  p_max =  1/4
EE_2d  = np.ravel(auto.EE);  EE_2d = EE_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[16,11]);  ax1 = fig2d.add_subplot(1, 1, 1)
ax1.set_ylabel('EE [km]');  ax1.set_title("Autocorr [" + data_name + "]");  ax1.set_xlabel(str_date)

# p1 = ax1.pcolormesh(num_1d, data.time[0][0]*3e5/2, EE_2d[:],       vmin=p_min, vmax=p_max, cmap='bwr')
p1  = ax1.pcolormesh(num_1d, data.time[0][0]*3e5/2, abs(EE_2d[:]),   norm=colors.LogNorm(vmin=p_min, vmax=p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('auto-corr')

plt.subplots_adjust(hspace=0.02); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-FT-EE-km.png'
    fig2d.savefig(png_fname)